<h3>HW9: Working with Pandas Dataframes</h3> 

Use the pandas module to answer the following questions about the EPA-HTTP data set.  Print the result of each part to the console.  Use pandas as much as you can; this includes the data structure and the analysis.

In [36]:
import re
import pandas as pd
import numpy as np
from StringIO import StringIO
import Tkinter
import tkFileDialog

In [45]:
def data_clean(data_file):
    """
    restructure the epa-http.txt file "HW9_data.txt" locally
    input: text file from local system selected using dialog box
    returns: pandas dataframe with and correct timestamp and quoting
    """

    pattern = r"(\" )(?=HTTP)"  # filter triple quoted lines

    with open(data_file) as d:
        raw = d.read()

    f_txt = re.sub(pattern, " ", raw) # removes the extra quote

    x = StringIO(f_txt) # load the data into StringIO

    df = pd.read_csv(x, sep="\s+", header=None, na_values="-")  # separate by space.
    df.columns = ['IP_address', 'time_stamp', 'request', 'status', 'bytes']

    df['time_stamp'] += "1995-08" # add year and month
    df['time_stamp'] = pd.to_datetime(df['time_stamp'], format="[%d:%H:%M:%S]%Y-%m")

    return df

In [38]:
# Select file using dialog box
root = Tkinter.Tk()
root.withdraw()
file_p = tkFileDialog.askopenfilename(parent=root)

In [39]:
# clean up the file using data_clean()
new_data = data_clean(file_p)

# check to make sure it's structured properly
print new_data[:5]

                IP_address          time_stamp  \
0            141.243.1.172 1995-08-29 23:53:25   
1  query2.lycos.cs.cmu.edu 1995-08-29 23:53:36   
2         tanuki.twics.com 1995-08-29 23:53:53   
3       wpbfl2-45.gate.net 1995-08-29 23:54:15   
4       wpbfl2-45.gate.net 1995-08-29 23:54:16   

                                     request  status  bytes  
0                GET /Software.html HTTP/1.0     200   1497  
1                GET /Consumer.html HTTP/1.0     200   1325  
2                    GET /News.html HTTP/1.0     200   1014  
3                             GET / HTTP/1.0     200   4889  
4  GET /icons/circle_logo_small.gif HTTP/1.0     200   2624  


In [40]:
# 1. Which hostname or IP address made the most requests?
print "IP_address or hostname with most requests:"
print new_data['IP_address'].value_counts()[:1]

IP_address or hostname with most requests:
sandy.rtptok1.epa.gov    294
dtype: int64


In [41]:
# 2. Which hostname or IP address received the most total bytes from the server? How many bytes did it receive?
ip_1 = new_data.groupby(new_data.IP_address)
byte_totals = ip_1['bytes'].aggregate(np.sum)
byte_totals.sort(inplace=True, ascending=False)
print "IP_address and hostname with most total bytes:"
print byte_totals[:1]

IP_address and hostname with most total bytes:
IP_address
piankhi.cs.hamptonu.edu    7267751
Name: bytes, dtype: float64


In [42]:
#3. During what hour was the server the busiest in terms of requests?
time_1 = new_data.groupby(new_data['time_stamp'].dt.hour)
group_size = time_1.size()
group_size.sort(inplace=True, ascending=False)
print "Busiest server hour | number of requests:"
print "                 " + str(group_size[:1]) 

Busiest server hour | number of requests:
                 14    4716
dtype: int64


In [43]:
#4 Which .gif image was downloaded the most during the day?
s = new_data[(new_data.status == 200) & (new_data.request.str.contains('\\.gif'))]
print "                Most downloaded gif image | download count"
print sub.request.value_counts()[:1]

                Most downloaded gif image | download count
GET /icons/circle_logo_small.gif HTTP/1.0    2465
dtype: int64


In [44]:
#5 What HTTP reply codes were sent other than 200?
http_s = new_data[new_data.status != 200]
print "Codes | counts:"
print http_s.status.value_counts()

Codes | counts:
304    5300
302    4506
404     611
501     272
403     272
500      69
400       6
dtype: int64
